In [ ]:
#@title Missing loops
#@markdown This notebook from [github.com/matteoferla/pyrosetta_help](https://github.com/matteoferla/pyrosetta_help)
#@markdown adds the missing parts of a given structure by cannibilising fragments of the AlphaFold2 model.

#@markdown As it operates by threading, the uniprot to be threaded can be of a close homologue as threading is actually 
#@markdown intended to be used. Do note that the alignment is done with Biopython's PairWise2, which 
#@markdown not really suitable for distant homologues.

#@markdown This is intended for aestetic purposes only as adding missing loops is bad and this approach is very quirky.
#@markdown The workings are explained in [this blog post](https://blog.matteoferla.com/2021/10/filling-missing-loops-by-cannibalising.html).


In [ ]:
#@title Installation
# ================================================================

#@markdown Modules installed pyrosetta

# ================================================================
#@markdown Installing PyRosetta with optional backup to your drive (way quicker next time!).
#@markdown Note that PyRosetta occupies some 10 GB, so you'll need to be on the 100 GB plan of Google Drive (it's one pound a month).

#@markdown NB. If `use_drive` is True, you will be prompted to give permission to
#@markdown use Google Drive —_always_ remember to check strangers code against data theft: search and look for all instances of `http`, `requests` and `post` in the code.

#@markdown ### Download PyRosetta
#@markdown The following is not the real password. However, the format is similar.
username = 'boltzmann'  #@param {type:"string"}
password = 'constant'  #@param {type:"string"}
#@markdown Are these the "normal" common credentials?
#@markdown If so their hash will be checked beforehand to check if they are correct
#@markdown (we don't want colab blocked by too many typos).
hash_comparision_required = True  #@param {type:"boolean"}
#@markdown The release to install will be the latest (from [graylab.jhu.edu/download/PyRosetta4](https://graylab.jhu.edu/download/PyRosetta4/archive/release/PyRosetta4.Release.python37.ubuntu/) for latest).

#@markdown Use Google Drive for PyRosetta (way faster next time, but takes up space)
#@markdown (NB. You may be prompted to follow a link and possibly authenticate and then copy a code into a box
use_drive = True  #@param {type:"boolean"}
#@markdown Installing `rdkit` and `rdkit_to_params` allows the creation of custom topologies (params) for new ligands
install_rdkit = True  #@param {type:"boolean"}

# ================================================================

import sys
import os
import importlib
import site
from typing import *


# ## Functions

def get_shell_mode() -> str:
    """
    Muppet-proofing: are we in colab?
    """
    # get_ipython is a standard libary in ipython, but not script
    from IPython import get_ipython
    shell_name = get_ipython().__class__.__name__
    if shell_name == 'Shell':
        return 'colab'
    elif shell_name == 'ZMQInteractiveShell':
        return 'jupyter'
    elif shell_name == 'TerminalInteractiveShell':
        raise RuntimeError('This is a colabs notebook. Why are you running it in the terminal?')
    else:
        raise RuntimeError(f'This is a colabs notebook. not a {shell_name}')


def assert_password(username: str, password: str) -> None:
    """
    Verify the username and password are correct without actually knowing them.
    I worry that there may be a large number of idiots that try to guess the passwords
    thus getting Colab fail2ban jailed.
    """
    import hashlib
    hashed_username = hashlib.sha256(username.encode()).hexdigest()
    hashed_password = hashlib.sha256(password.encode()).hexdigest()
    expected_hashed_username = 'cf6f296b8145262b22721e52e2edec13ce57af8c6fc990c8ae1a4aa3e50ae40e'
    expected_hashed_password = '45066dd976d8bf0c05dc8dd4d58727945c3437e6eb361ba9870097968db7a0da'
    msg = ('The hash of the {} is not as expected: ' +
           'if your username and password combo are correct and just not the academic ones, ' +
           'set `hash_comparision_required` to False.' +
           'If you dont know the password visit the Rosetta Commons site. ' +
           'The password and username for PyRosetta are DIFFERENT than Rosetta or FoldIt. ' +
           'Please do not google for username:password for this or for anything ' +
           'as stats like number of registered users is mighty important for grants and stuff.')
    assert hashed_username == expected_hashed_username, msg.format('username')
    assert hashed_password == expected_hashed_password, msg.format('password')


def install_and_import(package_name: str,
                       pypi_name: Optional[str] = None,
                       alias_name: Optional[str] = None):
    """If the module has a different name in pypi (`pypi_name`)
    than its import name (`package_name`), specify it.

         pip install pypi_name
         import package_name as alias_name
    """
    import importlib
    # I will go to hell for this, but shmeh:
    from pip._internal.cli.main import main as pip_main
    if pypi_name is None:
        pypi_name = package_name
    if alias_name is None:
        alias_name = package_name
    try:
        importlib.import_module(package_name)
    except ImportError as error:
        if error.name != package_name:
            # these are not the droids we are looking for
            raise ImportError(f'Import of {package_name} requires module {error.name}...',
                              name=error.name)
        pip_main(['install', pypi_name])
    globals()[alias_name] = importlib.import_module(package_name)



def set_workingpath():
    # ## Use drive?
    modality = get_shell_mode()
    if modality != 'colab':
        # jupyter --> stay
        return './'
    elif use_drive:
        from google.colab import drive
        drive.mount('/content/drive')
        return '/content/drive/MyDrive'
        os.chdir(_path)
    else:  # --> stay
        return '/content'

# ## Install pyrosetta
def get_latest_release_filename(username: str, password: str) -> str:
    # assumes the system is Ubuntu
    import sys, requests, re
    from IPython.display import display, HTML
    py_version = str(sys.version_info.major) + str(sys.version_info.minor)
    url_to_latest = f'https://graylab.jhu.edu/download/PyRosetta4/archive/release/PyRosetta4.Release.python{py_version}.ubuntu/latest.html'
    latest_response = requests.get(url_to_latest,
                                   auth=requests.auth.HTTPBasicAuth(username, password)
                                   )
    if latest_response.status_code == 401:
        raise ValueError('Incorrect username or password!')
    elif latest_response.status_code not in (200, 300, 301, 302, 303, 304, 305, 306, 307, 308):
        display(HTML(latest_response.text))
        raise ValueError(f'Something is wrong with the url {url_to_latest}')
    return re.search(r'[uU][rR][lL]=(.*?)["\s]', latest_response.text).group(1)

def install_pyrosetta(path:str):
    import sys, importlib, site
    # is pyrosetta installed?
    if importlib.util.find_spec('pyrosetta'):
        import pyrosetta
    # is there a Pyrosetta release file in the location?
    elif any(['PyRosetta4.Release' in filename for filename in os.listdir(path) if os.path.isdir(filename)]):
        release_folder = [filename for filename in os.listdir(path)
                          if 'PyRosetta4.Release' in filename and os.path.isdir(filename)][0]
        assert not os.system(f'pip3 install -e {_path}/{release_folder}/setup/')
    # download
    else:
        # check if hash is right.
        if hash_comparision_required:
            assert_password(username, password)
        # download tar and uncompress it on the fly
        latest_filename = get_latest_release_filename(username, password)
        assert not os.system(
            f'curl -u {username}:{password} {latest_filename} | tar -x')
        import re
        pyrosetta_folder = re.sub(r'.tar.\w+$', '', latest_filename)
        assert not os.system(f'pip3 install -e {_path}/{pyrosetta_folder}/setup/')
    # refresh:
    site.main()

import site
_path = set_workingpath()
install_pyrosetta(_path)
install_and_import(package_name='Bio', pypi_name='biopython')
if install_rdkit:
    install_and_import(package_name='rdkit', pypi_name='rdkit-pypi')
    # importing these in main makes them render properly in the notebook:
    from rdkit import Chem
    from rdkit.Chem import PandasTools
install_and_import(package_name='pyrosetta_help', pypi_name='pyrosetta-help', alias_name='ph')
install_and_import(package_name='rdkit_to_params', pypi_name='rdkit-to-params')
install_and_import('py3Dmol')
site.main()

In [ ]:
#@title Start PyRosetta
import pyrosetta
import pyrosetta_help as ph

#@markdown Do not optimise hydrogen on loading:
no_optH = False #@param {type:"boolean"}
#@markdown Ignore (True) or raise error (False) if novel residue (e.g. ligand) —  **don't tick this**.
ignore_unrecognized_res=False  #@param {type:"boolean"}
#@markdown Use autogenerated PDB residues are often weird (bad geometry, wrong match, protonated etc.): —best do it properly and parameterise it, so **don't tick this**.
load_PDB_components=False  #@param {type:"boolean"}
#@markdown Ignore all waters:
ignore_waters=True  #@param {type:"boolean"}

extra_options= ph.make_option_string(no_optH=no_optH,
                                  ex1=None,
                                  ex2=None,
                                  mute='all',
                                  ignore_unrecognized_res=ignore_unrecognized_res,
                                  load_PDB_components=load_PDB_components,
                                  ignore_waters=ignore_waters)


# capture to log
logger = ph.configure_logger()
pyrosetta.init(extra_options=extra_options)

In [ ]:
#@markdown ## Load template
pdb_code = '' #@param {type:"string"}

#@markdown Alternatively, leave blank to ypload PDB file of the template structure.
from google.colab import files

#@markdown If the template has novel ligands, they will be loaded too.
#@markdown But to do this a residue type  (=topology) needs to be made or loaded.
#@markdown These are saved as "params files".
#@markdown These following options control both the "acceptor" and "donor" poses (if uploaded).

#@markdown ### Params

#@markdown * Some compounds are parameterised in the database folder of rosetta,
#@markdown others in the PDB component database (if loaded).
#@markdown * Uses the params defined in the cell of the acceptor pose.
#@markdown * If there is no topology avalaible one will be made.
#@markdown * If a params file is present in the working folder it will use it.
#@markdown * See below or visit https://params.mutanalyst.com/ to generate them (upload the with the folder icon on the left).

#@markdown This forces it (a bit silly):
force_parameterisation = False  #@param {type:"boolean"}
#@markdown If it needs to be parameterised make it protonated for pH 7?
neutralise_params=True #@param {type:"boolean"}
save_params=True #@param {type:"boolean"}


#@markdown If a params file is present in the working folder it will use it.
#@markdown Leave this blank... otherwise  (comma separated w/ no rando spaces):
extra_params_files_to_use = '' #@param {type:"string"}
extra_params = [f for f in extra_params_files_to_use.split(',') if f]
use_all_folder_params= True  #@param {type:"boolean"}
if use_all_folder_params:
  present_params = [filename for filename in os.listdir() if os.path.splitext(filename) == '.params']
else:
  present_params = []
print('loading pose...')
if pdb_code:
  pdb_filename = ph.download_pdb(pdb_code)
  with open(pdb_filename, 'r') as fh:
    pdbblock = fh.read()
else:
  uploaded = files.upload()
  assert len(uploaded) ==1, 'wrong number of files (only one plz)'
  pdbblock = list(uploaded.values())[0]
template_pose = ph.parameterised_pose_from_pdbblock(pdbblock,
                                                                  wanted_ligands = [],
                                                                  force_parameterisation=force_parameterisation,
                                                                  neutralise_params=neutralise_params,
                                                                  save_params=save_params,
                                                                  overriding_params=extra_params+present_params)

In [ ]:
#@markdown ## Case A: Steal from AlphaFold2 (monomer)
#@markdown This section assumes the easy case of the presence of a single peptide,
#@markdown for the more complex case, skip and see the next cell.
#@markdown A uniprot accession is something like P08684.
#@markdown I am aware that RSCB PDB API does give back Uniprot ids,
#@markdown but it is too much faff and its simpler asking the user!

uniprot = '' #@param {type:"string"}
af_pose = ph.pose_from_alphafold2(uniprot)
fragsets = ph.make_fragment_sets(af_pose)
threaded, threader, threadites = ph.thread(target_sequence=af_pose.sequence(),
                                  template_pose=template_pose,
                                  fragment_sets=fragsets)
ph.steal_ligands(template_pose, threaded)
threaded.dump_pdb('threaded.pdb')

In [ ]:
#@markdown ## Case B: Steal from AlphaFold2 (complex)
#@markdown This section assumes the complex case of the presence of a multiple peptides (one per chain),
#@markdown Otherwise see previous cell.

#@markdown Write chain correspondence as a comma or semicolon separated series of chain colon uniprot, 
#@markdown e.g. `A:P63092,B:P62873,G:P59768`. Due to ambiguity spaces are ignored.
uniprots = '' #@param {type:"string"}
keep_unlisted = '' #@param {type:"boolean"}
chain2uniprot = dict([p.split(':') for p in uniprots.replace(' ', '').replace(';', ':').split(',')])
combo_threaded = pyrosetta.Pose()
clean_template_pose = template_pose.clone()
pyrosetta.rosetta.core.pose.remove_nonprotein_residues(clean_template_pose)
for chain in clean_template_pose.split_by_chain():
    pdb_info = chain.pdb_info()
    letter = pdb_info.chain(1)
    if letter not in chain2uniprot and keep_unlisted:
        pyrosetta.rosetta.core.pose.append_pose_to_pose(combo_threaded, chain, True)
    elif letter not in chain2uniprot:
        pass # byee
    uniprot = chain2uniprot[letter]
    af_pose = ph.pose_from_alphafold2(uniprot)
    fragsets = ph.make_fragment_sets(af_pose)
    threaded, threader, threadites = ph.thread(target_sequence=af_pose.sequence(),
                                      template_pose=chain,
                                      fragment_sets=fragsets)
    pyrosetta.rosetta.core.pose.append_pose_to_pose(combo_threaded, threaded, True)
ph.steal_ligands(template_pose, threaded)
threaded.dump_pdb('threaded.pdb')

In [ ]:
#@title Download PDB
files.download('threaded.pdb')

In [ ]:
#@title Upload to Michelanglo (optional)
#@markdown [Michelanglo](https://michelanglo.sgc.ox.ac.uk/) is a website that
#@markdown allows the creation, annotation and sharing of a webpage with an interactive protein viewport.
#@markdown ([examples](https://michelanglo.sgc.ox.ac.uk/gallery)).
#@markdown The created pages are private —they have a 1 in a quintillion change to be guessed within 5 tries.

#@markdown Registered users (optional) can add interactive annotations to pages.
#@markdown A page created by a guest is editable by registered users with the URL to it
#@markdown (this can be altered in the page settings).
#@markdown Leave blank for guest (it will not add an interactive description):

username = ''  #@param {type:"string"}
password = ''  #@param {type:"string"}

import os
assert not os.system(f'pip3 install michelanglo-api')
import site
site.main()
from michelanglo_api import MikeAPI, Prolink
if not username:
  mike = MikeAPI.guest_login()
else:
  mike = MikeAPI(username, password)
    
page = mike.convert_pdb(pdbblock=ph.get_pdbstr(nicked.acceptor_pose),
                        data_selection='ligand',
                        data_focus='residue',
                        )
if username:
     page.retrieve()
     ligands_prolink = Prolink(text='migrated ligand(s)',
                               focus='residue',
                               selection=' or '.join(wanted_ligands))
     
     page.description = '## Description\n\n'
     page.description += f'Model with {ligands_prolink} from {code} (chain {donor_chain}),'
     page.description += 'namely:\n\n'
     for lig in wanted_ligands:
         page.description += '* '+ Prolink(text=lig,
                                          focus='residue',
                                          selection=lig) +'\n'
     page.commit()
page.show_link()